Training an embedding with a simple dataset where the word relationships are very clear.

In [ ]:
# Needed to import from the data directories
import sys
import tensorflow as tf

from pathlib import Path

parent_dir = Path().resolve().parent
sys.path.append(str(parent_dir))

In [ ]:
from utils import import_dataset, merge_datasets, _pre_process_sentence, visualize_embeddings
from model import Word2Vec
from evaluation_data.poke_types import POKEMONS_BY_TYPE

In [5]:
easy = import_dataset('../train_data/evolutions_and_types.json')
hard = import_dataset('../train_data/pokedex_entries.json')

#dataset = merge_datasets(easy, hard[:1])
dataset = easy

In [6]:
window_size = 10
embedding_dim = 200

epochs = 100
# Use Adam optimizer with learning rate decay
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=1000,
        decay_rate=0.95
    )
)

# better with neg examples
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

word2vec = Word2Vec(dataset, embedding_dim)
train_dataset = word2vec.prepare_dataset(window_size=window_size, num_negative_samples=0)
word2vec.compile(optimizer, loss, metrics=['accuracy'])
word2vec.fit(train_dataset, epochs=epochs, verbose=1)

word2vec.summary()

Sample word pairs: [('bulbasaur', 'ivysaur'), ('bulbasaur', 'venusaur'), ('bulbasaur', 'grass'), ('bulbasaur', 'poison'), ('ivysaur', 'bulbasaur')]
Epoch 1/100


2025-03-17 07:17:48.971932: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-03-17 07:17:48.972125: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-17 07:17:48.972133: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-17 07:17:48.972262: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-17 07:17:48.972528: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-03-17 07:17:49.281179: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 1s 46ms/step - loss: 0.6889 - accuracy: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 0s 18ms/step - loss: 0.6433 - accuracy: 0.0000e+00
Epoch 3/100
5/5 [==============================] - 0s 17ms/step - loss: 0.6030 - accuracy: 0.0000e+00
Epoch 4/100
5/5 [==============================] - 0s 17ms/step - loss: 0.5659 - accuracy: 0.0089
Epoch 5/100
5/5 [==============================] - 0s 14ms/step - loss: 0.5320 - accuracy: 0.1036
Epoch 6/100
5/5 [==============================] - 0s 14ms/step - loss: 0.5022 - accuracy: 0.2857
Epoch 7/100
5/5 [==============================] - 0s 13ms/step - loss: 0.4764 - accuracy: 0.5000
Epoch 8/100
5/5 [==============================] - 0s 14ms/step - loss: 0.4544 - accuracy: 0.6554
Epoch 9/100
5/5 [==============================] - 0s 9ms/step - loss: 0.4359 - accuracy: 0.8000
Epoch 10/100
5/5 [==============================] - 0s 9ms/step - loss: 0.4204 - accuracy: 0.9018
Epoch 11/100
5/5 [===

In [ ]:
# Load and organize evaluation data

all_pokemons = []
for pokemons in POKEMONS_BY_TYPE.values():
    all_pokemons.extend(pokemons)

pokemon_to_type = {}
for type_name, pokemon_list in POKEMONS_BY_TYPE.items():
    for pokemon in pokemon_list:
        pokemon_to_type[pokemon] = type_name

poke_types = [pokemon_to_type[poke] for poke in all_pokemons]
poke_and_types = {poke: poke_type for poke, poke_type in zip(all_pokemons, poke_types)}

# Remove those that aren't in the vocabulary
pokes_to_remove = []
for pokemon in poke_and_types.keys():
    if not (pokemon in word2vec.vocabulary):
        pokes_to_remove.append(pokemon)         

for poke in pokes_to_remove:
    poke_and_types.pop(poke)

['charmander', 'charmeleon', 'charizard', 'vulpix', 'ninetales', 'growlithe', 'arcanine', 'ponyta', 'rapidash', 'magmar', 'flareon', 'moltres', 'squirtle', 'wartortle', 'blastoise', 'psyduck', 'golduck', 'poliwag', 'poliwhirl', 'poliwrath', 'tentacool', 'tentacruel', 'slowpoke', 'slowbro', 'seel', 'dewgong', 'shellder', 'cloyster', 'krabby', 'kingler', 'horsea', 'seadra', 'goldeen', 'seaking', 'staryu', 'starmie', 'magikarp', 'gyarados', 'lapras', 'bulbasaur', 'ivysaur', 'venusaur', 'oddish', 'vileplume', 'paras', 'parasect', 'bellsprout', 'weepinbell', 'victreebel', 'exeggcute', 'exeggutor', 'tangela', 'pikachu', 'raichu', 'magnemite', 'magneton', 'voltorb', 'electrode', 'electabuzz', 'jolteon', 'zapdos', 'abra', 'kadabra', 'alakazam', 'drowzee', 'hypno', 'mr.mime', 'mewtwo', 'mew', 'pidgey', 'pidgeotto', 'pidgeot', 'rattata', 'raticate', 'jigglypuff', 'wigglytuff', 'meowth', 'persian', 'snorlax']


In [12]:
visualize_embeddings(word2vec, dim=2, words=poke_and_types, title='Word2Vec Pokembeddings',) 

In [ ]:
from evaluation_tests.test_types import test_script

In [ ]:
results = test_script(word2vec)

Pass: 99.3% (596 of 600)
 Fail: 0.7% (4 of 600)


In [ ]:
results.fail_counter

2

In [ ]:
# Save the model to a directory
model_save_path = "./saved_model"
word2vec.save(model_save_path)

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets
